In [1]:
import sys
sys.path.append("clean/RecSys/code/")  # for lightning env
sys.path.append("../RecSys/code/")  # locally

import warnings
warnings.filterwarnings("ignore")  # going to depreciate warnings, provided conda env works

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dataclasses import dataclass

@dataclass
class ArgsVal:
    dataset: str = "demo"
    batch_size: int = 1
    num_workers: int = 4
    T: int = 4  # number of previous articles to consider in the prompt
    datafraction: float = 1.0  # how much of entire dataset to use
    old: bool = True  # load checkpoints for model and not via modelwrapper
    from_checkpoint: str = "clean/model_0.0001.pth"  # path to checkpoint locally
    model: str = "CG"  # conditional generation
    backbone: str = "google/mt5-small"
    tokenizer: str = "google/mt5-small"
    evaltrain: bool = False  # allows for evaluation on training set

@dataclass
class ArgsTrain(ArgsVal):
    evaltrain: bool = True  # allows for evaluation on training set

argsval = ArgsVal()
argstrain = ArgsTrain()

In [4]:
# load model
from models.get_model import get_model

model_val = get_model(argsval)
model_train = get_model(argstrain)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


<All keys matched successfully>
<All keys matched successfully>


In [5]:
# choose the prompt to use
# the provided checkpoints was trained using subtitles
from prompt_templates import create_prompt_subtitles

In [7]:
!ls

clean  demo_data  main.py


In [8]:
# %cd clean/RecSys/code
%cd ../code

/teamspace/studios/this_studio/clean/RecSys/code


In [9]:
# create the dataset
from dataloader import EkstraBladetDataset

# split sets output form of model
data_val = EkstraBladetDataset(argsval, create_prompt_subtitles, split="validation")
data_train = EkstraBladetDataset(argstrain, create_prompt_subtitles, split="validation")

In [10]:
example_data = next(iter(data_val))
type(example_data), example_data.keys()

(dict, dict_keys(['prompts', 'targets', 'categories']))

In [11]:
example_data["prompts"]  # all prompts for the inview articles

['En bruger har for nylig læst artikler: Andelen af elever med høj trivsel er faldet til 87,4 procent, Heftig debat om at leve uden sex. Mange - både kvinder og mæ, Fra 2026 skal Honda levere motorer til Aston Martin, som der, Planen lød blandt andet på at angribe fængselsbetjente med p, vil brugeren læse artiklen En kinesisk forsker har måske afsløret, at Kina tog teori om laboratorielæk mere alvorligt end hidtil antaget? (ja/nej)\n',
 'En bruger har for nylig læst artikler: Andelen af elever med høj trivsel er faldet til 87,4 procent, Heftig debat om at leve uden sex. Mange - både kvinder og mæ, Fra 2026 skal Honda levere motorer til Aston Martin, som der, Planen lød blandt andet på at angribe fængselsbetjente med p, vil brugeren læse artiklen Kursen på den svenske krone har stort set aldrig været så lav, som den er nu. Det kan danskerne drage nytte af. Ekstra Bladet har undersøgt, hvor de store besparelser kan findes? (ja/nej)\n',
 'En bruger har for nylig læst artikler: Andelen af 

In [12]:
example_data["targets"]  # showing which article was clicked on (1)

[0, 0, 0, 0, 0, 1]

In [13]:
# previous clicked categories
example_data["categories"][:argsval.T]

['nyheder', 'nationen', 'sport', 'nyheder']

In [14]:
# categories of inview articles
example_data["categories"][argsval.T:]

['nyheder', 'forbrug', 'forbrug', 'underholdning', 'nyheder', 'sport']

In [15]:
# the model cannot process lists of strings
# the collator tokenizes these inputs and creates tensors
from collators import CollatorUnderstand
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained(argsval.tokenizer)
collator = CollatorUnderstand(tokenizer)
dl_val = DataLoader(data_val, batch_size=1, collate_fn=collator, shuffle=False)
dl_train = DataLoader(data_train, batch_size=1, collate_fn=collator, shuffle=False)

In [16]:
# prompt_input_ids: tokenized prompts
# prompt_attention_mask: make each element in the batch as long as the longest tokenized prompt, mask padding tokens
# decoder_start: tokenized 'ja / nej' for CG model
next(iter(dl_val))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'prompt_input_ids': tensor([[  642, 36428,   588,  ...,     0,     0,     0],
         [  642, 36428,   588,  ...,     0,     0,     0],
         [  642, 36428,   588,  ...,     0,     0,     0],
         [  642, 36428,   588,  ...,     0,     0,     0],
         [  642, 36428,   588,  ...,  2794,   271,     1],
         [  642, 36428,   588,  ...,     0,     0,     0]]),
 'prompt_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'decoder_start': tensor([[ 432,  259,  275, 3810,    1],
         [ 432,  259,  275, 3810,    1],
         [ 432,  259,  275, 3810,    1],
         [ 432,  259,  275, 3810,    1],
         [ 432,  259,  275, 3810,    1],
         [ 432,  259,  275, 3810,    1]]),
 'targets': [0, 0, 0, 0, 0, 1],
 'categories': ['nyheder',
  'nationen',
  'sport',
  'nyheder',
  'nyheder',
  'forbrug',

In [17]:
import numpy as np

In [18]:
it_val = iter(dl_val)

In [20]:
example = next(it_val)
scores = model_val.validation_step(example).cpu().numpy()
targets = np.array(example["targets"])
prompts = example["prompts"]

# targets ranked according to model scores
order = np.argsort(scores)[::-1]
score_ranked_targets = np.take(targets, order)
scores, targets, prompts, score_ranked_targets

(array([0.51089793, 0.06474407, 0.03934271, 0.16496603, 0.62189466,
        0.02306686, 0.3466548 ], dtype=float32),
 array([0, 0, 0, 0, 0, 0, 1]),
 ['En bruger har for nylig læst artikler: Andelen af elever med høj trivsel er faldet til 87,4 procent, Heftig debat om at leve uden sex. Mange - både kvinder og mæ, Fra 2026 skal Honda levere motorer til Aston Martin, som der, Planen lød blandt andet på at angribe fængselsbetjente med p, vil brugeren læse artiklen ? (ja/nej)\n',
  'En bruger har for nylig læst artikler: Andelen af elever med høj trivsel er faldet til 87,4 procent, Heftig debat om at leve uden sex. Mange - både kvinder og mæ, Fra 2026 skal Honda levere motorer til Aston Martin, som der, Planen lød blandt andet på at angribe fængselsbetjente med p, vil brugeren læse artiklen Korsfæstelser, afhuggede hoveder og krigshandlinger var i årevis dagligdag for to mænd, som er idømt lange straffe for landsforræderi. Men de er ikke de eneste i familien, der er dømt for at støtte Islam

In [21]:
it_train = iter(dl_train)

In [22]:
example = next(it_train)
scores = model_train.validation_step(example).cpu().numpy()
targets = np.array(example["targets"])
prompts = example["prompts"]

# targets ranked according to model scores
order = np.argsort(scores)[::-1]
score_ranked_targets = np.take(targets, order)
scores, targets, prompts, score_ranked_targets

(array([0.49515206, 0.5216051 , 0.42004386, 0.66117924, 0.54520535,
        0.71452135, 0.5508101 , 0.03665067, 0.5397415 , 0.2359618 ,
        0.7062714 ], dtype=float32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 ['En bruger har for nylig læst artikler: Anklagemyndighed ønsker grundlaget for fængsling udvidet i s, Politiet skred onsdag aften til anholdelse af en mand i Fæll, Konen efter afdøde Jørgen Haugen Sørensen, der var en af lan, Konen efter afdøde Jørgen Haugen Sørensen, der var en af lan, vil brugeren læse artiklen David Beckham havde både gave og rosende ord med til kong Charles, da de to forleden mødtes til en modebegivenhed i London? (ja/nej)\n',
  'En bruger har for nylig læst artikler: Anklagemyndighed ønsker grundlaget for fængsling udvidet i s, Politiet skred onsdag aften til anholdelse af en mand i Fæll, Konen efter afdøde Jørgen Haugen Sørensen, der var en af lan, Konen efter afdøde Jørgen Haugen Sørensen, der var en af lan, vil brugeren læse artiklen Går turen syd